In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import one_hot
from torchsummary import summary
from model import OCR

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
pathToData = 'data1'
os.listdir(path=pathToData)

['sample_submission.csv', 'test.csv', 'train.csv']

In [4]:
trainingData = pd.read_csv(os.path.join(pathToData,'train.csv'))

In [5]:
trainingData

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
Y_train = trainingData['label']
X_train = trainingData.drop(labels = ["label"],axis = 1) 

In [7]:
Y_train.value_counts()

label
1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: count, dtype: int64

nói chung đoạn này cho thấy ảnh không bị imbalance

In [8]:
#reshape lại ảnh
def arr2img(arr):
    img = np.array(X_train.iloc[0])
    return img.reshape((28,28))

show ảnh ra cho vui

In [41]:

img = arr2img(X_train.iloc[0])
plt.imshow(img,cmap='gray')
plt.title(trainingData.iloc[0,0])
plt.axis("off")
plt.show()

AttributeError: 'Tensor' object has no attribute 'iloc'

In [10]:
X_train = np.array([arr2img(i) for i in X_train])
Y_train = np.array(Y_train)

In [11]:
#chuẩn bị dữ liệu
X_train = torch.from_numpy(X_train)
X_train = X_train.float()
X_train = torch.unsqueeze(X_train, 1)#thêm 1 chiều cho dữ liệu
# X_train = X_train.to(device)
X_train[0].shape

torch.Size([1, 28, 28])

In [34]:
Y_train = torch.from_numpy(Y_train)
Y_train = Y_train.float()
Y_train = one_hot(Y_train, num_classes=11).to(device=device)
Y_train


RuntimeError: one_hot is only applicable to index tensor.

In [13]:
X_train[0].shape

torch.Size([1, 28, 28])

In [25]:
#summary
model = OCR()
summary(model=model,input_size=(X_train[0].shape))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 16, 28, 28]           2,320
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           4,640
            Conv2d-5           [-1, 32, 14, 14]           9,248
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 64, 7, 7]          18,496
            Conv2d-8             [-1, 64, 7, 7]          36,928
         MaxPool2d-9             [-1, 64, 3, 3]               0
           Linear-10                       [-1]         295,168
             ReLU-11                       [-1]               0
           Linear-12                       [-1]           2,827
Total params: 369,787
Trainable params: 369,787
Non-trainable params: 0
-------------------------------

In [46]:
# model = OCR()
# model.eval()
# model(X_train[0])
model.train()
# Y_train[0]

OCR(
  (conv1a): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1b): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2a): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2b): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3a): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.4, inplace=False)
  (dense1): Linear(in_features=576, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=11, bias=True)
  (relu): ReLU()
)

In [33]:
model.to(device=device)
X_train.to(device=device)
model(X_train[0])
# Y_train[0].requires_grad


False

In [39]:
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
crit = torch.nn.MSELoss(reduction='mean')
sample_size = X_train.shape[0]
Y_train = Y_train.to(device=device)
from tqdm import tqdm
for t in tqdm(range(200), ncols = 100):
    for i in range(sample_size):
        opt.zero_grad(sample_size)
        y_pred = model(X_train[i])   #x is tensor of independent vars
        loss = crit(y_pred,Y_train[i]) #y is tensor of outcomes
        loss.backward()
        opt.step()

  0%|                                                                       | 0/200 [00:00<?, ?it/s]

c:\Users\ADMIN\Downloads\src code\python\Bib-Number-Recognition\model.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return softmax(features)
 60%|████████████████████████████████████▎                        | 119/200 [05:50<03:58,  2.95s/it]


KeyboardInterrupt: 